In [9]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import src.model as models
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

In [10]:
VERSION = "2"
MODEL_BASE_NAME = "MegaClassifier_a_MobileNetV3"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [11]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [12]:
OPTIMIZERS = {
    "Adam": tf.keras.optimizers.legacy.Adam(),
}

In [13]:
LOSS = {
    "BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy(),
    "BinaryFocalCrossentropy": tf.keras.losses.BinaryFocalCrossentropy(
        alpha=0.25, gamma=2.0
    ),
    "SigmoidFocalCrossEntropy": tfa.losses.SigmoidFocalCrossEntropy(
        alpha=0.25, gamma=2.0
    ),
    "FocalLoss": models.FocalLoss(alpha=0.25, gamma=2.0),
}

In [14]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

In [15]:
BATCH_SIZE = 64

In [16]:
SUBVERSION = 0
for LOSS_NAME, _ in LOSS.items():
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    REPORTS_PATH = os.path.abspath(
        f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )

    MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    )
    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validationtrain_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.MobileNetV3Large(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS["Adam"],
        loss=LOSS[LOSS_NAME],
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
    )

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    accuracy_chart = graphics.create_training_accuracy_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    accuracy_chart.write_image(f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.png")

    loss_chart = graphics.create_training_loss_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    loss_chart.write_image(f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.png")

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(MODELS_PATH)

    results = model.evaluate(test_images)
    metric_names = history.model.metrics_names
    evaluation_results = {
        ("test_" + name): value for name, value in zip(metric_names, results)
    }

    evaluation = pd.DataFrame([evaluation_results])
    evaluation.to_csv(
        os.path.join(LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.csv"),
        sep=";",
        index=False,
    )

    y_pred_prob = model.predict(test_images)
    y_true = test_images.labels

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    roc_curve_chart = graphics.create_roc_curve_chart(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        model_name=MODEL_COMPLETE_NAME,
    )
    roc_curve_chart.write_image(f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.png")

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # y_pred_prob = y_pred_prob.flatten()
    y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    # Crear el gráfico correctamente
    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.png"
    )

    y_pred_class = (y_pred_prob > 0.5).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.png"
    )

    SUBVERSION += 1
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Epoch 1/10


2025-03-01 17:12:04.036765: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - ETA: 0s - loss: 0.2992 - accuracy: 0.8674 - precision: 0.9006 - recall: 0.9222 - auc: 0.9256

2025-03-01 17:13:39.396166: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


280/280 [==============================] - 119s 417ms/step - loss: 0.2992 - accuracy: 0.8674 - precision: 0.9006 - recall: 0.9222 - auc: 0.9256 - val_loss: 0.2553 - val_accuracy: 0.9232 - val_precision: 0.9366 - val_recall: 0.9482 - val_auc: 0.9627
Epoch 2/10
280/280 [==============================] - 156s 557ms/step - loss: 0.1966 - accuracy: 0.9210 - precision: 0.9443 - recall: 0.9491 - auc: 0.9702 - val_loss: 0.2089 - val_accuracy: 0.9316 - val_precision: 0.9507 - val_recall: 0.9457 - val_auc: 0.9712
Epoch 3/10
280/280 [==============================] - 137s 490ms/step - loss: 0.1787 - accuracy: 0.9277 - precision: 0.9500 - recall: 0.9522 - auc: 0.9752 - val_loss: 0.1873 - val_accuracy: 0.9316 - val_precision: 0.9469 - val_recall: 0.9499 - val_auc: 0.9762
Epoch 4/10
280/280 [==============================] - 148s 528ms/step - loss: 0.1683 - accuracy: 0.9321 - precision: 0.9540 - recall: 0.9540 - auc: 0.9780 - val_loss: 0.1797 - val_accuracy: 0.9319 - val_precision: 0.9383 - val_reca

...

280/280 [==============================] - 113s 395ms/step - loss: 0.0149 - accuracy: 0.9098 - precision: 0.9452 - recall: 0.9293 - auc: 0.9671 - val_loss: 0.0139 - val_accuracy: 0.9232 - val_precision: 0.9219 - val_recall: 0.9658 - val_auc: 0.9764
Epoch 2/10
280/280 [==============================] - 102s 366ms/step - loss: 0.0105 - accuracy: 0.9329 - precision: 0.9553 - recall: 0.9539 - auc: 0.9801 - val_loss: 0.0123 - val_accuracy: 0.9356 - val_precision: 0.9588 - val_recall: 0.9432 - val_auc: 0.9763
Epoch 3/10
280/280 [==============================] - 108s 387ms/step - loss: 0.0099 - accuracy: 0.9381 - precision: 0.9599 - recall: 0.9562 - auc: 0.9824 - val_loss: 0.0118 - val_accuracy: 0.9319 - val_precision: 0.9527 - val_recall: 0.9439 - val_auc: 0.9786
Epoch 4/10
280/280 [==============================] - 110s 393ms/step - loss: 0.0094 - accuracy: 0.9395 - precision: 0.9601 - recall: 0.9580 - auc: 0.9840 - val_loss: 0.0118 - val_accuracy: 0.9361 - val_precision: 0.9651 - val_reca

INFO:tensorflow:Assets written to: /Users/carlos/WORKSPACE/MegaClassifier/models/MegaClassifier_a_MobileNetV3/v2/v2.3/assets


67/67 [==============================] - 21s 306ms/step - loss: 0.0115 - accuracy: 0.9321 - precision: 0.9718 - recall: 0.9242 - auc: 0.9819


2025-03-01 18:30:13.798224: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


67/67 [==============================] - 21s 297ms/step



